In [1]:
!git clone https://github.com/kumargopal/BgChanger.git

fatal: destination path 'BgChanger' already exists and is not an empty directory.


In [2]:
# prompt: mount gdrive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd BgChanger

/content/BgChanger


In [4]:
!pip install "jax[cuda12_local]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [5]:
!pip install flask-cloudflared

In [6]:

import os
import numpy as np
from flask import Flask, request, jsonify, send_file, render_template_string

from PIL import Image
import io
import matplotlib.pyplot as plt
from BgRemover.bg_remover import BgRemover
from BgReplacer.StableDiffusionInPaint import StableDiffusionInPaint

# Initialize Flask app
app = Flask(__name__)

# Define paths and model files
model_path = "/content/gdrive/My Drive"
restore_model = "isnet.pth"
background_remover = BgRemover(model_path, restore_model)

inpaint_model_path = "runwayml/stable-diffusion-inpainting"
inpaint_model = StableDiffusionInPaint(inpaint_model_path)

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
text_encoder/pytorch_model.fp16.safetensors not found


Fetching 24 files:   0%|          | 0/24 [00:00<?, ?it/s]

In [7]:
# HTML template
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Image Inpainting</title>
</head>
<body>
    <h1>Upload an Image and Enter a Prompt</h1>
    <form action="/process" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*" required><br><br>
        <label for="prompt">Prompt:</label>
        <input type="text" id="prompt" name="prompt" required><br><br>
        <button type="submit">Generate</button>
    </form>
    {% if result_imgs %}
    <h2>Output Images:</h2>
    {% for img in result_imgs %}
        <img src="{{ img }}" alt="Output Image"><br>
    {% endfor %}
    {% endif %}
</body>
</html>
'''

In [11]:
@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)




@app.route('/process', methods=['POST'])
def process():
    file = request.files['file']
    prompt = request.form['prompt']

    # Save the uploaded image to Google Drive
    image_path = '/content/gdrive/My Drive/uploaded_image.png'
    original_image = Image.open(file.stream)
    original_image.save(image_path)

    print("Image saved at:", image_path)
    print("Original image type:", type(original_image))

    # Load and process the image
    result_image = background_remover.remove_background(image_path)
    result_np = np.array(result_image)

    # Create mask image (white background, black foreground)
    mask_np = np.all(result_np[:, :, :3] == 0, axis=-1).astype(np.uint8) * 255
    mask_pil = Image.fromarray(mask_np)

    # Generate inpainted image
    result_image = inpaint_model.inpaint(result_image, mask_pil, prompt)

    # Save the result to a BytesIO object
    img_io = io.BytesIO()
    result_image.save(img_io, 'PNG')
    img_io.seek(0)

    return send_file(img_io, mimetype='image/png')



AssertionError: View function mapping is overwriting an existing endpoint function: index

In [9]:
from flask_cloudflared import run_with_cloudflared

In [10]:
run_with_cloudflared(app)
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/flask_cloudflared.py", line 142, in start_cloudflared
    cloudflared_address = _run_cloudflared(port, metrics_port, tunnel_id, config_path)
  File "/usr/local/lib/python3.10/dist-packages/flask_cloudflared.py", line 137, in _run_cloudflared
    raise Exception(f"! Can't connect to Cloudflare Edge")
Exception: ! Can't connect to Cloudflare Edge


In [ ]:
import os
import numpy as np
from flask import Flask, request, send_file, render_template
from PIL import Image
import io
from BgRemover.bg_remover import BgRemover
from BgReplacer.StableDiffusionInPaint import StableDiffusionInPaint

# Initialize Flask app
app = Flask(__name__)

# Define paths and model files
model_path = "/content/gdrive/My Drive"
restore_model = "isnet.pth"
background_remover = BgRemover(model_path, restore_model)

inpaint_model_path = "runwayml/stable-diffusion-inpainting"
inpaint_model = StableDiffusionInPaint(inpaint_model_path)

import zipfile

@app.route('/')
def index():
    return send_file('FrontEnd/index.html')

@app.route('/process', methods=['POST'])
def process():
    file = request.files['file']
    prompt = request.form['prompt']

    # Save the uploaded image to Google Drive
    image_path = '/content/gdrive/My Drive/uploaded_image.png'
    original_image = Image.open(file.stream)
    original_image.save(image_path)

    print("Image saved at:", image_path)
    print("Original image type:", type(original_image))

    # Load and process the image
    result_image = background_remover.remove_background(image_path)
    result_np = np.array(result_image)

    # Create mask image (white background, black foreground)
    mask_np = np.all(result_np[:, :, :3] == 0, axis=-1).astype(np.uint8) * 255
    mask_pil = Image.fromarray(mask_np)

    # Generate inpainted images
    result_images = inpaint_model.inpaint(result_image, mask_pil, prompt)

    # Save result images to a zip file
    zip_io = io.BytesIO()
    with zipfile.ZipFile(zip_io, 'w') as zip_file:
        for i, img in enumerate(result_images):
            img_io = io.BytesIO()
            img.save(img_io, 'PNG')
            img_io.seek(0)
            zip_file.writestr(f'result_image_{i+1}.png', img_io.read())
    zip_io.seek(0)

    return send_file(zip_io, mimetype='application/zip', as_attachment=True, download_name='result_images.zip')


from flask_cloudflared import run_with_cloudflared

run_with_cloudflared(app)
app.run()
